In [1]:
import tensorflow as tf
from tensorflow import keras
import collections
from datasets import load_dataset
import datasets
import math

2021-09-21 16:51:03.692662: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-21 16:51:03.692760: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
word_index = keras.datasets.imdb.get_word_index()
inverted_word_index = dict((i, word) for (word, i) in word_index.items())

dataset = datasets.load_dataset('imdb')

Reusing dataset imdb (/home/leherlemaxime/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
file = open("vader_lexicon.txt", "r", encoding="utf-8")
lines = file.readlines()
file.close()

good_lines = []

for line in lines:
    line_temp = line.replace("\t", " ")
    line_temp = line_temp.split(" ")
    line_temp = line_temp[:2]
    good_lines.append(line_temp)
    
dict_value = dict(good_lines)

In [4]:
def clean_the_text(text_array):
    
    specialChars = "()\\\''.,;:\"?-" 
    for specialChar in specialChars:
        text_array = text_array.replace(specialChar, ' ')
        
    text_array = text_array.replace("/>", ' ')
    text_array = text_array.replace("<br", ' ')
    text_array = text_array.replace("!", " ! ")
    
    text_array = text_array.split(" ")
    
    result_array = [elem.lower() for elem in text_array if(len(elem) != 0)]
    
    return result_array

In [5]:
def dataset_to_array(dataset):
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    
    for elem in dataset["train"]:
        y_train.append(elem["label"])
        x_train.append(clean_the_text(elem["text"]))
        
    for elem in dataset["test"]:
        y_test.append(elem["label"])
        x_test.append(clean_the_text(elem["text"]))
        
    return x_train, y_train, x_test, y_test

In [6]:
x_train, y_train, x_test, y_test = dataset_to_array(dataset)

In [39]:
def word_array_to_feature(word_array):
    feature = []
    
    # No feature
    
    if ("no" in word_array):
        feature.append(1)
    else:
        feature.append(0)
        
    # Pronouns feature
    
    valid_pronouns = ["i", "me", "my", "mine", "myself", "you", "your", "yours", "yourself", "we", "us", "our", "ourselves"]
    pronouns_count = 0
    for elem in word_array:
        if (elem in valid_pronouns):
            pronouns_count += 1
            
    feature.append(pronouns_count)
            
    # ! feature
        
    if ("!" in word_array):
        feature.append(1)
    else:
        feature.append(0)
        
    # log(nb_word) feature
    
    feature.append(math.log(len(word_array)))
    
    # positive and negative feature
    positive_count = 0
    negative_count = 0
    
    for elem in word_array :
        if ((elem in dict_value) and (float(dict_value[elem]) >= 1.5)):
            positive_count += 1
        elif ((elem in dict_value) and (float(dict_value[elem]) <= -1.5)):
            negative_count += 1
            
    feature.append(positive_count)
    feature.append(negative_count)
    
    return feature

In [40]:
x_train_feature = [word_array_to_feature(elem) for elem in x_train]
x_test_feature = [word_array_to_feature(elem) for elem in x_test]

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support

In [42]:
clf = LogisticRegression(random_state=0).fit(x_train_feature, y_train)

In [43]:
y_pred = clf.predict(x_test_feature)

In [44]:
precision_recall_fscore_support(y_test, y_pred)

(array([0.71007589, 0.70627874]),
 array([0.7036 , 0.71272]),
 array([0.70682311, 0.70948475]),
 array([12500, 12500]))